In [15]:
import pandas as pd
import pickle 
from scipy.stats import poisson

In [16]:
# Cargar el archivo Pickle en la variable dict_table
dict_table = pickle.load(open('dict_table', 'rb'))

# Ahora puedes usar dict_table sin problemas
dict_table['Group H']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Portugal,0,0,0,0,0,0,0,0
1,2,Ghana,0,0,0,0,0,0,0,0
2,3,Uruguay,0,0,0,0,0,0,0,0
3,4,South Korea,0,0,0,0,0,0,0,0


In [17]:
df_historical_data=pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture=pd.read_csv('clean_fifa_worldcup_fixture.csv')

In [21]:
df_fixture

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


# 1 Calcular Team Strenght

In [24]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
791,Russia,Croatia,2018,2,2,4
792,France,Belgium,2018,1,0,1
793,Croatia,England,2018,2,1,3
794,Belgium,England,2018,2,0,2


In [26]:
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [28]:
df_home

,HomeTeam,HomeGoals,AwayGoals
0,France,4,1
1,Argentina,1,0
2,Chile,3,0
3,Chile,1,0
4,Argentina,6,3
...,...,...,...
791,Russia,2,2
792,France,1,0
793,Croatia,2,1
794,Belgium,2,0


In [30]:
df_home = df_home.rename(columns = {'HomeTeam':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away = df_away.rename(columns = {'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'})


In [32]:
df_home

,Team,GoalsScored,GoalsConceded
0,France,4,1
1,Argentina,1,0
2,Chile,3,0
3,Chile,1,0
4,Argentina,6,3
...,...,...,...
791,Russia,2,2
792,France,1,0
793,Croatia,2,1
794,Belgium,2,0


In [34]:
df_away

,Team,GoalsConceded,GoalsScored
0,Mexico,4,1
1,France,1,0
2,Mexico,3,0
3,France,1,0
4,Mexico,6,3
...,...,...,...
791,Croatia,2,2
792,Belgium,1,0
793,England,2,1
794,England,2,0


In [36]:
# Concatenar los DataFrames
df_team_strenght = pd.concat([df_home, df_away], ignore_index=True)

# Agrupar solo por 'Team' y obtener el primer valor de cada grupo
df_team_strenght = df_team_strenght.groupby('Team').mean()

# Mostrar el resultado
print(df_team_strenght)


              GoalsScored  GoalsConceded
Team                                    
Algeria          1.000000       1.461538
Angola           0.333333       0.666667
Argentina        1.771429       1.185714
Australia        0.812500       1.937500
Austria          1.576923       1.692308
...                   ...            ...
Uruguay          1.634615       1.326923
Wales            0.800000       0.800000
West Germany     2.109091       1.309091
Yugoslavia       1.678571       1.285714
Zaire            0.000000       4.666667

[84 rows x 2 columns]


# 2 Funcion predict_points

In [39]:
def predict_points(home,away):
    if home in df_team_strenght.index and away in  df_team_strenght.index:
        #goals_scored * goals_conceded
        lamb_home = df_team_strenght.at[home,'GoalsScored'] * df_team_strenght.at[away,'GoalsConceded']
        lamb_away = df_team_strenght.at[away,'GoalsScored'] * df_team_strenght.at[home,'GoalsConceded']
        prob_home , prob_away , prob_draw = 0, 0, 0
        for x in range (0,11):#number of goals home team
            for y in range(0,11):#number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y,lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y :
                    prob_home += p
                else:
                    prob_away += p
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return(points_home,points_away)
    else:
        return (0,0)

## 2.1 Testeo de la funcion

In [42]:
predict_points('Angola','Austria')

(0.9209040398327188, 1.743703220048267)

# 3 Prediciendo Mundial

## 3.1 Fase de grupo

In [46]:
# dividiendo fixture en grupo , octavos , cuartos , semifinal , final
df_fixture_group48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [48]:
df_fixture_group48 

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
5,Netherlands,Match 36,Qatar,2022
6,England,Match 3,Iran,2022
7,United States,Match 4,Wales,2022
8,Wales,Match 17,Iran,2022
9,England,Match 20,United States,2022


In [50]:
df_fixture_final


,home,score,away,year
62,Losers Match 61,Match 63,Losers Match 62,2022
63,Winners Match 61,Match 64,Winners Match 62,2022


In [52]:
for group in dict_table:
    print(dict_table[group]['Team'].values)

['Qatar (H)' 'Ecuador' 'Senegal' 'Netherlands']
['England' 'Iran' 'United States' 'Wales']
['Argentina' 'Saudi Arabia' 'Mexico' 'Poland']
['France' 'Australia' 'Denmark' 'Tunisia']
['Spain' 'Costa Rica' 'Germany' 'Japan']
['Belgium' 'Canada' 'Morocco' 'Croatia']
['Brazil' 'Serbia' 'Switzerland' 'Cameroon']
['Portugal' 'Ghana' 'Uruguay' 'South Korea']


In [54]:
for group in dict_table:
    dict_table[group]['Pts'] = 0
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group48[df_fixture_group48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home,away = row ['home'],row['away']
        points_home,points_away = predict_points(home,away)
        dict_table[group].loc[dict_table[group]['Team'] == home,'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away,'Pts'] += points_away
    dict_table[group] = dict_table[group].sort_values('Pts' , ascending = False).reset_index()
    dict_table[group] = dict_table[group][['Team','Pts']]
    dict_table[group] = dict_table[group].round(0)

C:\Users\Erick\AppData\Local\Temp\ipykernel_12968\2592395203.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.806048]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home,'Pts'] += points_home
C:\Users\Erick\AppData\Local\Temp\ipykernel_12968\2592395203.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.39834848]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home,'Pts'] += points_home
C:\Users\Erick\AppData\Local\Temp\ipykernel_12968\2592395203.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.87662202]' has dtype incompatible with int64, ple

In [55]:
for group in dict_table:
    dict_table[group]['Pts'] = 0
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group48[df_fixture_group48['home'].isin(teams_in_group)]
    
    print(f"Partidos para el grupo {group}:")
    print(df_fixture_group_6[['home', 'away']])

    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)

        print(f"Predicción para el partido {home} vs {away}: {points_home}-{points_away}")

        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)
    print(f"Tabla final del grupo {group}:")
    print(dict_table[group])


C:\Users\Erick\AppData\Local\Temp\ipykernel_12968\1572758176.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.806048]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home


Partidos para el grupo Group A:
          home         away
1      Senegal  Netherlands
3  Netherlands      Ecuador
4      Ecuador      Senegal
5  Netherlands        Qatar
Predicción para el partido Senegal vs Netherlands: 0.8060480036913967-1.998819489994606
Predicción para el partido Netherlands vs Ecuador: 2.08584988803822-0.7057715668851557
Predicción para el partido Ecuador vs Senegal: 1.1947181862346672-1.553002815761932
Predicción para el partido Netherlands vs Qatar: 0-0
Tabla final del grupo Group A:
          Team  Pts
0  Netherlands  4.0
1      Senegal  2.0
2      Ecuador  2.0
3    Qatar (H)  0.0
Partidos para el grupo Group B:
             home           away
6         England           Iran
7   United States          Wales
8           Wales           Iran
9         England  United States
10          Wales        England
11           Iran  United States
Predicción para el partido England vs Iran: 2.3983484773716217-0.42638326026943146
Predicción para el partido United State

C:\Users\Erick\AppData\Local\Temp\ipykernel_12968\1572758176.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.39834848]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Erick\AppData\Local\Temp\ipykernel_12968\1572758176.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.87662202]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Erick\AppData\Local\Temp\ipykernel_12968\1572758176.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.1926621]' has dtype incompatible with int6

Predicción para el partido Wales vs England: 1.0777690524296295-1.6016764515950142
Predicción para el partido Iran vs United States: 0.8760740735936343-1.906459041458192
Tabla final del grupo Group B:
            Team  Pts
0        England  6.0
1          Wales  5.0
2  United States  4.0
3           Iran  2.0
Partidos para el grupo Group C:
            home          away
12     Argentina  Saudi Arabia
13        Mexico        Poland
14        Poland  Saudi Arabia
15     Argentina        Mexico
16        Poland     Argentina
17  Saudi Arabia        Mexico
Predicción para el partido Argentina vs Saudi Arabia: 2.876622022782785-0.06013942498656184
Predicción para el partido Mexico vs Poland: 0.8310293375382896-1.9789145789451226
Predicción para el partido Poland vs Saudi Arabia: 2.7665692083598206-0.16024489518781027
Predicción para el partido Argentina vs Mexico: 2.373718985593784-0.48929711431609224
Predicción para el partido Poland vs Argentina: 0.978447965326555-1.825162897602601
Predi

C:\Users\Erick\AppData\Local\Temp\ipykernel_12968\1572758176.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.43491855]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home


Predicción para el partido Tunisia vs Australia: 1.5934437681097382-1.1684734946443591
Predicción para el partido France vs Denmark: 1.748319347825104-1.0556690478598059
Predicción para el partido Australia vs Denmark: 0.4642855217418519-2.3942534773618096
Predicción para el partido Tunisia vs France: 0.40992988425551813-2.4593706967067446
Tabla final del grupo Group D:
        Team  Pts
0     France  7.0
1    Denmark  6.0
2    Tunisia  3.0
3  Australia  2.0
Partidos para el grupo Group E:
          home        away
24     Germany       Japan
25       Spain  Costa Rica
26       Japan  Costa Rica
27       Spain     Germany
28       Japan       Spain
29  Costa Rica     Germany
Predicción para el partido Germany vs Japan: 2.4349185488215572-0.42555894883462325
Predicción para el partido Spain vs Costa Rica: 2.090205947477715-0.725337920347499
Predicción para el partido Japan vs Costa Rica: 1.3892094992164359-1.366689654984112
Predicción para el partido Spain vs Germany: 0.9134758750836108

C:\Users\Erick\AppData\Local\Temp\ipykernel_12968\1572758176.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.79286069]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Erick\AppData\Local\Temp\ipykernel_12968\1572758176.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.130412]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home


Predicción para el partido Croatia vs Belgium: 1.8515969794212046-0.9567510850566354
Predicción para el partido Canada vs Morocco: 0.21471117234169726-2.5705755434456945
Tabla final del grupo Group F:
      Team  Pts
0  Croatia  7.0
1  Belgium  5.0
2  Morocco  4.0
3   Canada  0.0
Partidos para el grupo Group G:
           home         away
36  Switzerland     Cameroon
37       Brazil       Serbia
38     Cameroon       Serbia
39       Brazil  Switzerland
40       Serbia  Switzerland
41     Cameroon       Brazil
Predicción para el partido Switzerland vs Cameroon: 2.130412004220575-0.6689622811285773
Predicción para el partido Brazil vs Serbia: 2.4778797208200496-0.37739670103886447
Predicción para el partido Cameroon vs Serbia: 0.9946434181860102-1.6684342275571162
Predicción para el partido Brazil vs Switzerland: 2.4991283181672737-0.38787486046522485
Predicción para el partido Serbia vs Switzerland: 1.0762963297926385-1.6748811678477686
Predicción para el partido Cameroon vs Brazil: 0.

C:\Users\Erick\AppData\Local\Temp\ipykernel_12968\1572758176.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.4132101]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home


Predicción para el partido South Korea vs Portugal: 0.3927322015991233-2.488925811396529
Tabla final del grupo Group H:
          Team  Pts
0     Portugal  6.0
1      Uruguay  6.0
2        Ghana  4.0
3  South Korea  2.0


In [56]:
dict_table['Group A']

,Team,Pts
0,Netherlands,4.0
1,Senegal,2.0
2,Ecuador,2.0
3,Qatar (H),0.0


## 3.2 Octavos de Final

In [61]:
#Actualizar el fixture con el 1 puesto (group winner) y 2 puesto (runners up)
for group in dict_table:
    group_winner = dict_table[group].loc[0,'Team']
    runners_up = dict_table[group].loc[1,'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                f'Runners-up {group}':runners_up},inplace=True)

df_fixture_knockout['winner']='?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [63]:
#Crear funcion get_winner
def get_winner(df_fixture_updated):
    for index,row in df_fixture_updated.iterrows():
        home,away = row['home'],row['away']
        points_home,points_away = predict_points(home,away)
        if points_home > points_away:
            winner=home
        else:
            winner = away
        df_fixture_updated.loc[index,'winner'] = winner
    return df_fixture_updated

In [65]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Croatia
55,Portugal,Match 56,Switzerland,2022,Portugal


## 3.3 Cuartos de Final

In [68]:
def update_table (df_fixture_round_1,df_fixture_round_2):
    for index ,row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index,'winner']
        match = df_fixture_round_1.loc[index,'score']
        df_fixture_round_2.replace({f'Winners {match}':winner},inplace= True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [70]:
update_table(df_fixture_knockout,df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Croatia,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [72]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Croatia,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


## 3.4 Semifinal

In [74]:
update_table(df_fixture_quarter,df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [83]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


## 3.5 Final

In [78]:
update_table(df_fixture_semi,df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [80]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
